In [1]:
from pathlib import Path

import numpy as np
from scipy import stats
from shapely.geometry import Point
import geopandas

In [2]:
# https://math.stackexchange.com/q/444700/593484
# See alternative method 1: https://corysimon.github.io/articles/uniformdistn-on-sphere/
NUM_WORLD_LOCATIONS = 37067
# in miles:
MEAN_RADIUS_OF_EARTH = 3958.7613
DATA_PATH = Path("../data/")


In [34]:
def cartesian_to_gcs(v):
    """Change the basis used to represent
    a *unit* vector in R^3 (3-dimensional Euclidean space)
    from the standard basis for R^3 into a basis
    for a geographical coordinate system (GCS).  Output is
    in degrees. Only the latitude and longitude are returned
    (in that order).

    Args:
        v: numpy.ndarray

    Returns:
        numpy.ndarray
    """
    # https://en.wikipedia.org/wiki/Spherical_coordinate_system#In_geography
    x = v[0]
    y = v[1]
    z = v[2]
    # Assume v has L-2 norm of 1.
    r = 1.0
    # https://stackoverflow.com/questions/50906886/convert-spherical-to-geographical-coordinate-system
    # Calculate the latitude
    theta = np.arccos(z / r) * 180.0 / np.pi
    latitude = theta - 90.0
    # Calculate the longitude
    phi = np.sign(y) * np.arccos(x / np.sqrt(x**2 + y**2)) * 180.0 / np.pi
    longitude = phi
    return np.array([latitude, longitude])

def get_uniformly_distributed_points_on_sphere(n:int):
    """The points are returned with respect to the
    standard basis for R^3 as an numpy.ndarray.
    x, y, and z-coordinates are in the 0, 1, and 2 columns,
    respectively. 
    Each row represents a different point.

    References:
    https://math.stackexchange.com/q/444700/593484
    See alternative method 1: https://corysimon.github.io/articles/uniformdistn-on-sphere/
    https://gis.stackexchange.com/a/356502
    """
    # The user desires n points.  However, it is hard to generate
    # exactly n points in a numerically stable way.  Therefore,
    # we try to generate int(n * n_safety) points.
    vecs_on_sphere = np.empty(shape=(n, 3))
    n_safety = 1.01
    num_points_generated = 0

    while num_points_generated < n:
        # Generate int(n * n_safety) vectors.
        # Store these vectors as columns in a matrix V
        # for further manipulation.
        # Only take those columns with a large enough norm.
        V = stats.norm.rvs(size=(int(n * n_safety), 3))
        norms_of_V_rows = np.linalg.norm(x=V, ord=2, axis=1, keepdims=False)
        good_rows_of_V = norms_of_V_rows > 0.0001

        # https://stackoverflow.com/a/49763264/8423001
        num_points_generated = np.count_nonzero(good_rows_of_V)
        if num_points_generated < n:
            n_safety = n_safety + 0.01
            # continue with next iteration of loop
            continue
        # We found the number of vectors required by n
        # that meet our numerical requirement of being long enough.
        # Save them.
        vecs = V[good_rows_of_V, :][0:n, :] 
        norms_of_vecs = norms_of_V_rows[good_rows_of_V][0:n]
        # https://stackoverflow.com/questions/19602187/numpy-divide-each-row-by-a-vector-element
        vecs_on_sphere = vecs / norms_of_vecs.reshape(-1, 1)
        # The vectors can now
        # be stretched so that they land on the unit-sphere.
        
    return vecs_on_sphere

    # np.apply_along_axis(
    #     func1d=cartesian_to_gcs,
    #     axis=0,
    #     arr=vecs_on_sphere
    # )

def get_central_angle(n1, n2):
    """Given the so-called n-vectors n1 and n2,
    find the central angle between them.
    
    Args:
        n1: numpy.ndarray
        n2: numpy.ndarray

    See: https://en.m.wikipedia.org/wiki/N-vector
    """
    delta_sigma = np.arctan2(
        np.linalg.norm(
            np.abs(np.cross(n1, n2)), 
            ord=2
        ),
        np.dot(n1, n2)
    )

    return delta_sigma

def great_circle_dist(p, q, r):
    """Given points p and q represented
    as n-vectors on a sphere
    with radius r, find the great-circle
    distance between them.

    Args:
        p: numpy.ndarray
        q: numpy.ndarray

    See: https://en.m.wikipedia.org/wiki/N-vector
    """
    delta_sigma = get_central_angle(n1=p, n2=q)
    d = r * delta_sigma

    return d

def great_circle_dist_matrix(vecs, r):
    """Given a set of n-vectors on a sphere of
    radius r, construct the symmetric matrix 
    showing all of their
    pair-wise great-circle distances.
    For simplicity, garbage is returned along
    the main diagonal and the lower triangular matrix.
    Only refer to the upper triangular matrix of the
    output.

    Args:
        vecs: numpy.ndarray. This should be a 
        matrix with each row representing an
        n-vector. It should have 3 columns
        for the x, y, and z coordinates of
        each point.
    Returns:
        numpy.ndarray
    """
    num_vecs = vecs.shape[0]
    dist_mat = np.empty(shape=(num_vecs, num_vecs))

    # Fill upper triangle.
    for i in range(num_vecs):
        for j in range(i + 1, num_vecs, 1):
            # Get distance for ith row and jth col.
            dist_mat[i, j] = great_circle_dist(p=vecs[i], q=vecs[j], r=r)

    return dist_mat

def n_vector_to_lat_long(v):
    """Convert n-vectors to their two-dimensional
    representation on a sphere.
    
    Converts to latitudes and longitudes.

    Args:
        v: numpy.ndarray.  This should be a matrix
        with rows representing n-vectors and columns
        representing x, y, and z coordinates.

    Returns:
        numpy.ndarray with two columns and the same
        number of rows as v.  The 1st column is the
        latitude and the 2nd col is the longitude.

    See: https://en.m.wikipedia.org/wiki/N-vector
    """
    n_x = v[0]
    n_y = v[1]
    n_z = v[2]
    latitude = np.arctan2(n_z / np.sqrt(n_x**2 + n_y**2))
    longitude = np.arctan2(n_y / n_x)



# Test Functions

In [55]:
my_points = get_uniformly_distributed_points_on_sphere(3)
print(f"my_points: \n", my_points)

my_points: 
 [[-0.02586324  0.99380063  0.1081268 ]
 [ 0.80855533  0.57040455  0.14448851]
 [-0.41383937 -0.82778741  0.37882051]]


In [56]:
great_circle_dist_matrix(vecs=my_points, r=1)

array([[0.02586324, 0.97450268, 2.45119334],
       [0.80855533, 0.57040455, 2.42196398],
       [0.41383937, 0.82778741, 0.37882051]])

In [ ]:
# https://en.m.wikipedia.org/wiki/Great-circle_distance
# Before loop:
# Fix a point on the unit-sphere.
fixed_point = vecs_on_sphere[:, 0]

delta_sigma = np.empty(shape=vecs_on_sphere.shape[1] - 1)
# Get a vector of greatest-circle
# distances to the other points.
for j in range(1, vecs_on_sphere.shape[1]):
    delta_sigma[j - 1] = np.arctan2(
        np.linalg.norm(np.cross(fixed_point, vecs_on_sphere[:, j]), ord=2),
        np.dot(fixed_point, vecs_on_sphere[:, j])
    )

In [ ]:
# Now, scale to the size of the Earth.
distances = MEAN_RADIUS_OF_EARTH * np.abs(delta_sigma)

In [ ]:
distances.sort()
delta_sigma.sort()

In [ ]:
distances[NUM_WORLD_LOCATIONS - 10: ]

In [ ]:
# https://github.com/SciTools/cartopy/issues/1325
cartopy.config["data_dir"] = Path(DATA_PATH, "raw_data")

In [ ]:
# https://datascience.stackexchange.com/questions/112104/sampling-from-earths-landmass
natural_earth_data = shapereader.natural_earth(
    resolution="10m",
    category="physical",
    name="land"
)

In [ ]:
print(natural_earth_data)

In [ ]:
land_on_earth = shapely.union_all(
    geometries=list(shapereader.Reader(natural_earth_data).geometries())
)

shapely.prepare(
    geometry=land_on_earth
)

In [ ]:
land_on_earth.contains(shapely.geometry.Point(33.792978, -84.412071))

In [ ]:
ecoregions_2017 = geopandas.read_file(filename=Path(DATA_PATH, "raw_data", "Ecoregions2017"))

In [ ]:
ecoregions_2017.crs

In [ ]:
ecoregions_2017.plot()

In [ ]:
ecoregions_2017.head

In [ ]:
# Points have longitude first and latitude second.
ecoregions_2017["geometry"].representative_point()

In [ ]:
ecoregions_2017["geometry"]

In [ ]:
additional_points_on_earth = geopandas.points_from_xy(x=np.array([27.099020]), y=np.array([37.640693]), crs="EPSG:4326")

In [ ]:
ecoregions_2017_geometry = ecoregions_2017["geometry"]

In [ ]:
ecoregions_2017_geometry_union = ecoregions_2017_geometry.unary_union